In [2]:
import pandas as pd
import numpy as np
import catboost
import sys
import os
sys.path.append(os.path.abspath('..'))
import utils
import scoring
import lightgbm
import xgboost
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
import itertools

%matplotlib inline

In [3]:
data = pd.read_hdf("../data/all_train_data.hdf")
target_labels = pd.read_hdf("../data/train_labels.hdf")

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
# Apply inplace
def preprocess_data(data: pd.DataFrame):
    new_columns = list(itertools.chain(*[[f'ClosestHit_{f}[{i}]' for i in range(4)] for f in ["X", "Y", "T", "z", "dx", "dy"]]))
    rename_dict = dict(zip(range(24), new_columns))
    data.rename(rename_dict, axis='columns', inplace=True)
    
    for j in range(4):
        distance_to_center = 0
        mh_distance_to_center = 0
        for i in ["X", "Y"]:
            data[f'Lextra_ClosestHit_dt_{i}[{j}]'] = np.square(data[f'ClosestHit_{i}[{j}]'] - data[f'Lextra_{i}[{j}]'])
            data[f'MatchedHit_ClosestHit_dt{i}[{j}]'] = np.square(data[f'ClosestHit_{i}[{j}]'] - data[f'MatchedHit_{i}[{j}]'])
            data[f'MatchedHit_Lextra_dt{i}[{j}]'] = np.square(data[f'Lextra_{i}[{j}]'] - data[f'MatchedHit_{i}[{j}]'])
            distance_to_center += np.square(data[f'ClosestHit_{i}[{j}]'])
            mh_distance_to_center += np.square(data[f'MatchedHit_{i}[{j}]'])                                     
        data[f'ClosestHit_to_Center[{j}]'] = np.sqrt(distance_to_center)
        data[f'MachtedHit_to_Center[{j}]'] = np.sqrt(mh_distance_to_center)
    
    

In [ ]:
preprocess_data(data)

In [7]:
train_x, test_x, train_y, test_y, = train_test_split(data, target_labels, test_size=0.4)


In [15]:
train_y.loc[train_y.weight < 0, 'label'] = 1 - train_y.loc[train_y.weight < 0, 'label'] # Invert labels
train_y.weight = train_y.weight.abs() # Take absolute weights


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [16]:
lgbm = lightgbm.LGBMClassifier(num_leaves=60, n_estimators=200, max_depth=7, n_jobs=-1)
catb = catboost.CatBoostClassifier(n_estimators=800, depth=7, thread_count=12)
lgbm2 = lightgbm.LGBMClassifier(num_leaves=31, n_estimators=50,
                                n_jobs=-1, boosting_type="dart", learning_rate=0.3)
xgbt = xgboost.XGBClassifier(n_estimators=250, max_depth=7, n_jobs=12)
models = [lgbm, catb, lgbm2, xgbt]

In [33]:
train_x.columns = train_x.columns.str.replace("[", "").str.replace("]", "")

In [ ]:
xgbt.fit(train_x, train_y.label, sample_weight=train_y.weight)

In [ ]:
# for model in models:
#     print(f"{model.__class__} is training.")
#     model.fit(train_x, train_y.label, sample_weight=train_y.weight)

In [ ]:
del train_x, train_y

## Stacking part

In [ ]:
# Define meta classifier
meta_clf = lightgbm.LGBMClassifier(num_leaves=5, n_estimators=200, learning_rate=0.1, n_jobs=-1)


Cross validate it

In [ ]:
k =  KFold(n_splits=5)
test_scores = []
train_scores = []
for train_index, test_index in k.split(test_x, test_y):
    
    mf_train_x, mf_test_x = test_x.iloc[train_index], test_x.iloc[test_index]
    mf_train_y, mf_test_y = test_y.iloc[train_index].copy(), test_y.iloc[test_index]
    
    # Invert labels & take absolute weights
    mf_train_y.loc[mf_train_y.weight < 0, 'label'] = 1 - mf_train_y.loc[mf_train_y.weight < 0, 'label'] 
    mf_train_y.weight = mf_train_y.weight.abs()

    
    meta_features = np.array([m.predict_proba(mf_train_x)[:,1] for m in models], axis=1).T
    print(meta_features.shape)
    break
    meta_clf.fit(meta_features, mf_train_y.label, sample_weight = mf_train_y.weight)
    
   
    meta_features_test = np.array([m.predict_proba(mf_test_x)[:,1] for m in models], axis=1).T

    
    train_preds = meta_clf.predict_proba(meta_features)[:, 1]
    train_score = 10000 * scoring.rejection90(np.array(mf_train_y.label),
                                      train_preds,
                                      sample_weight=np.array(mf_train_y.weight))
    train_scores.append(train_score)
    
    test_preds = meta_clf.predict_proba(meta_features_test)[:, 1]
    test_score = 10000* scoring.rejection90(np.array(mf_test_y.label),
                                       test_preds,
                                       sample_weight=np.array(mf_test_y.weight)) 
    test_scores.append(test_score)
    
    print(f"Train score: {train_score:.2f}, test score: {score:.2f}")

print(f"Train mean score: {np.mean(train_scores):.2f}, test mean score: {np.mean(scores):.2f}")




Check overfitting and score


In [ ]:
all_meta_features = np.array([m.predict_proba(test_x)[:,1] for m in models], axis=1).T

test_y.loc[test_y.weight < 0, 'label'] = 1 - test_y.loc[test_y.weight < 0, 'label'] 
test_y.weight = test_y.weight.abs()
    
mf_lgbm.fit(all_meta_features, test_y.label, sample_weight = test_y.weight)

# Make prediction

In [ ]:
# Make prediction
submission_data = pd.read_hdf("../data/all_test_data.hdf")
submission_data.rename(rename_dict, axis='columns', inplace=True)

preprocess_data(submission_data)


In [ ]:
submission_data.columns = submission_data.columns.str.replace("[", "").str.replace("]", "")

In [ ]:
submission_meta_features = np.array([m.predict_proba(submission_data)[:,1] for m in models], axis=1).T


In [ ]:
submission_y = meta_clf.predict_proba(submission_meta_features)[:, 1]

In [ ]:
pd.DataFrame(data={"prediction": submission_y}, index=submission_data.index).to_csv(
    "stacking_lgbm_lgbm_catboost_xgboost.csv", index_label="id")